In [1]:
import pickle
import time
import torch
import numpy as np
from argparse import ArgumentParser
from double_net.double_net import DoubleNet, train_loop, test_loop_random_start, test_loop
from double_net import datasets as ds
import double_net.plot_utils as pu
import matplotlib.pyplot as plt

In [2]:
train = True
test = True

In [3]:
# import logging
# logger = logging.getLogger()
# logger.setLevel(logging.INFO)

In [4]:
if torch.cuda.is_available():
    device = 'cuda'
else:
    device = 'cpu'

In [5]:
device

'cuda'

In [6]:
parser = ArgumentParser()
parser.add_argument('--random-seed', type=int, default=0)
parser.add_argument('--num-examples', type=int, default=131072 * 4)
parser.add_argument('--test-num-examples', type=int, default=10000)
parser.add_argument('--test-iter', type=int, default=5)
parser.add_argument('--n-agents', type=int, default=2)
parser.add_argument('--n-items', type=int, default=15)
parser.add_argument('--num-epochs', type=int, default=100)
parser.add_argument('--batch-size', type=int, default=128 * 32)
parser.add_argument('--test-batch-size', type=int, default=10000)
parser.add_argument('--model-lr', type=float, default=1e-3)
parser.add_argument('--misreport-lr', type=float, default=1e-1)
parser.add_argument('--misreport-iter', type=int, default=25)
parser.add_argument('--test-misreport-iter', type=int, default=2000)
parser.add_argument('--rho', type=float, default=1.0)
parser.add_argument('--rho-incr-iter', type=int, default=2)
parser.add_argument('--rho-incr-amount', type=float, default=1.0)
parser.add_argument('--lagr-update-iter', type=int, default=100)
parser.add_argument('--rgt-start', type=int, default=0)
parser.add_argument('--sinkhorn-epsilon', type=float, default=5e-2)
# parser.add_argument('--sinkhorn-rounds', type=int, default=20)
parser.add_argument('--sinkhorn-tol', type=float, default=0.01)

dataset_name = ""
args = parser.parse_args(args=[])
args

Namespace(batch_size=4096, lagr_update_iter=100, misreport_iter=25, misreport_lr=0.1, model_lr=0.001, n_agents=2, n_items=15, num_epochs=100, num_examples=524288, random_seed=0, rgt_start=0, rho=1.0, rho_incr_amount=1.0, rho_incr_iter=2, sinkhorn_epsilon=0.05, sinkhorn_tol=0.01, test_batch_size=10000, test_iter=5, test_misreport_iter=2000, test_num_examples=10000)

In [7]:
torch.manual_seed(args.random_seed)
np.random.seed(args.random_seed)

item_ranges = ds.preset_valuation_range(args.n_agents, args.n_items, dataset_name)
clamp_op = ds.get_clamp_op(item_ranges)

model = DoubleNet(
    args.n_agents, args.n_items, item_ranges, args.sinkhorn_epsilon, args.sinkhorn_tol, marginal_choice='exact_one'
).to(device)

train_data = ds.generate_dataset_nxk(args.n_agents, args.n_items, args.num_examples, item_ranges).to(device)
train_loader = ds.Dataloader(train_data, batch_size=args.batch_size, shuffle=True)

In [8]:
import os
dir_name = 'scaled_exact_one_2x15_experiment_data/'
os.makedirs(dir_name, exist_ok=True)

In [9]:

if train:
    start_time = time.time()
    mean_rgt, mean_pay, lagr_mults = train_loop(model, train_loader, args, device=device)
    end_time = time.time()
    with open(dir_name+'time.txt', 'w') as f:
        print(f"runtime: {end_time - start_time}", file=f)

100%|██████████| 100/100 [1:29:28<00:00, 53.68s/it]


{'regret_max': 0.5093187093734741, 'regret_mean': 0.040091633796691895, 'regret_mults': tensor([[6.4529, 5.9612]], device='cuda:0'), 'payment': 0.4265807271003723}
{'regret_max': 0.20464026927947998, 'regret_mean': 0.02392435260117054, 'regret_mults': tensor([[9.2654, 8.2034]], device='cuda:0'), 'payment': 0.6144001483917236}
{'regret_max': 0.20578396320343018, 'regret_mean': 0.017328141257166862, 'regret_mults': tensor([[11.7373, 10.8054]], device='cuda:0'), 'payment': 0.6115469932556152}
{'regret_max': 0.23646116256713867, 'regret_mean': 0.013297148048877716, 'regret_mults': tensor([[16.8106, 15.9583]], device='cuda:0'), 'payment': 0.6330505609512329}
{'regret_max': 0.23961418867111206, 'regret_mean': 0.01094242837280035, 'regret_mults': tensor([[19.1598, 19.0065]], device='cuda:0'), 'payment': 0.6315922737121582}
{'regret_max': 0.2593592405319214, 'regret_mean': 0.010120093822479248, 'regret_mults': tensor([[22.7057, 23.2262]], device='cuda:0'), 'payment': 0.6108881235122681}
{'regr

In [10]:
if not train:
    model = DoubleNet.load(dir_name).cuda()
else:
    model.save(dir_name)
    to_pkl_lst = [mean_rgt, mean_pay, lagr_mults]
    for i, fname in enumerate(['mean_rgt', 'mean_pay', 'lagr_mults']):
        with open(dir_name + fname, 'wb') as fp:
            pickle.dump(to_pkl_lst[i], fp)

In [11]:
%%time
# model.sinkhorn_rounds = 100
# model.sinkhorn_epsilon = 1e-2
if test:
    test_data = ds.generate_dataset_nxk(args.n_agents, args.n_items, args.test_num_examples, item_ranges).to(device)
    # test_data = torch.load(dir_name + 'test_data').to(device=device)
    cpu_test_data = test_data.clone().to(device='cpu')

    test_loader = ds.Dataloader(test_data, batch_size=args.test_batch_size, shuffle=True)

    result = test_loop(model, test_loader, args, device=device)
    print(result)

1it [00:30, 30.81s/it]


{'payment_mean': 0.5707309246063232, 'regret_mean': 0.002022291999310255, 'regret_max': 0.1868128925561905}
CPU times: user 30.8 s, sys: 16 ms, total: 30.9 s
Wall time: 30.8 s


In [12]:
# %%time
if test:
    args.test_num_examples = 1000
    args.test_batch_size = 1000
    args.test_misreport_iter = 1000
    args.misreport_lr = 1e-1
    test_data2 = ds.generate_dataset_nxk(args.n_agents, args.n_items, args.test_num_examples, item_ranges).to(device)
    random_starts = [test_data2]
    for i in range(10):
        random_starts.append(ds.generate_dataset_nxk(args.n_agents, args.n_items, args.test_num_examples, item_ranges).to(device))
    test_loader = ds.Dataloader(test_data2, batch_size=args.test_batch_size, shuffle=True)

    result = test_loop_random_start(model, test_loader, args, random_starts, device=device)
    print(result)

1it [02:47, 167.79s/it]


{'payment_mean': 0.5703034996986389, 'regret_mean': 0.0028555241879075766, 'regret_max': 0.18862362205982208}


In [13]:
converged = model.check_convergence(torch.rand(10000,2,3).cuda())

In [14]:
converged[0].max()

tensor(0.0007, device='cuda:0')

In [15]:
converged[1].max()

tensor(2.3842e-07, device='cuda:0')

In [16]:
model.cuda()
model.sinkhorn_epsilon=5e-3
if test:
    args.test_num_examples = 1000
    args.test_batch_size = 1000
    args.test_misreport_iter = 1000
    args.misreport_lr = 1e-1
    test_data2 = ds.generate_dataset_nxk(args.n_agents, args.n_items, args.test_num_examples, item_ranges).to(device)
    random_starts = [test_data2]
    for i in range(10):
        random_starts.append(ds.generate_dataset_nxk(args.n_agents, args.n_items, args.test_num_examples, item_ranges).to(device))
    test_loader = ds.Dataloader(test_data2, batch_size=args.test_batch_size, shuffle=True)

    result = test_loop_random_start(model, test_loader, args, random_starts, device=device)
    print(result)

1it [09:55, 595.61s/it]


{'payment_mean': 0.7524938583374023, 'regret_mean': 0.23816587030887604, 'regret_max': 0.408466100692749}
